In [1]:
import pandas as pd

In [44]:
temp = pd.read_csv('prof_info.csv',header=None)

In [45]:
temp = temp[temp[6]==' ']

In [46]:
new = temp.iloc[:,0:2]
new.drop_duplicates(inplace=True)

In [47]:
new.iloc[:,0:2].to_csv('failure.csv',index=False,header=False)

In [22]:
import os
import time
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from multiprocessing.dummy import Pool as ThreadPool
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
from functools import partial
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By    
from selenium.webdriver.support import expected_conditions as EC

In [23]:
def element_exits(xpath):
    
    try:
        driver.find_element_by_xpath(xpath)

        return True

    except:

        return False
    

In [160]:
df_fail = pd.read_csv('failure.csv',header=None)
df_fail.columns=['Name','Web']
web_list_fl, prof_name_list_fl = df_fail['Web'].to_list(), df_fail['Name'].to_list()
new_df_fail = df_fail.copy(deep=True)


In [161]:
new_df_fail

,Name,Web
0,Erik Cambria,https://scholar.google.com/citations?user=ilSY...
1,Wooi-Boon Goh,https://scholar.google.com.sg/citations?user=w...
2,Cai Wentong,https://scholar.google.com/citations?user=QdmI...
3,Trevor Erik CARLSON,https://scholar.google.com/citations?user=bUK5...
4,Jie Zhang,https://scholar.google.com/citations?hl=en&use...
5,Arnab Bhattacharyya,https://scholar.google.com/citations?hl=en&use...
6,Cristina Carbunaru,https://scholar.google.com/citations?hl=en&use...
7,Anand BHOJAN,https://scholar.google.com/citations?hl=en&use...
8,Cham Tat Jen,https://scholar.google.com/citations?user=Lx3X...
9,Owen Noel Newton Fernando,https://scholar.google.com/citations?hl=en&use...


In [162]:
def failure_crawler(ind):
    web,prof_name = web_list_fl[ind], prof_name_list_fl[ind]
    paper_list = []
    citation_list = []
    year_list = []
    author_list = []
    discription_list = []
    
    
    chrome_options = Options()
    chrome_options.add_argument("--headless") 
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=chrome_options)
    driver.maximize_window()
    driver.get(web)
    
    print('Crawling google scholar website of Porf:{}'.format(prof_name))
    
    research_area = driver.find_element_by_xpath('//div[@id="gsc_prf_int"]').text
    print(research_area)
    if not research_area:
        research_area = ' '
    try:
        
        time.sleep(0.5)
        #year
        for year in driver.find_elements_by_xpath('//span[@class="gsc_a_h gsc_a_hc gs_ibl"]'):
            
            year_list.append(year.text)
        
        #citation
        for citation in driver.find_elements_by_xpath('//a[@class="gsc_a_ac gs_ibl"]'):
            #print(citation.text)
            citation_list.append(citation.text)
       
        
        #paper author discription   
        for each_element in driver.find_elements_by_xpath('//a[@class="gsc_a_at"]'):
            #print(each_element.text)
            paper_list.append(each_element.text)
            
            python_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="gsc_a_b"]/tr[1]/td[1]/a')))
            driver.execute_script("arguments[0].scrollIntoView();", each_element)
            driver.execute_script("arguments[0].click();", each_element)
            
            
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//div[@id="gsc_vcd_table"]')))
            try:
                author = driver.find_element_by_xpath('//div[contains(text(),"Authors")]/following::*').text
            except:
                author = prof_name
            author_list.append(author)
         
            try:
                discription = driver.find_element_by_xpath('//*[@id="gsc_vcd_descr"]/div').text
                
            except:
                discription = 'None'
            discription_list.append(discription)
            #print(discription)
              
            close_button = driver.find_element_by_xpath('//*[@id="gs_md_cita-d-x"]/span[1]')
            driver.execute_script("arguments[0].scrollIntoView();", close_button)
            driver.execute_script("arguments[0].click();", close_button)
            time.sleep(1)
        
        if len(year_list) == len(citation_list) and len(paper_list)==len(year_list):
            temp_df = pd.DataFrame(
            {
                'Name': [prof_name]*len(year_list),
                'Google_Link' :[web]*len(year_list),
                'Paper': paper_list,
                'Author': author_list,
                'Citation': citation_list,
                'Year': year_list,
                'Discription': discription_list
            }
            )
            temp_df.to_csv('./prof_info.csv',index = False, mode= 'a',header= False)
            print('Successfully Crawling!')
            new_df_fail=new_df_fail.drop(new_df_fail[new_df_fail['Name']==prof_name].index)
        else:
            print('长度错了')
            print('Crawling Failure!(Store failure info)')

    except:
            print('元素抓取报错')
            print('Crawling Failure!(Store failure info)')                                                                             
    driver.quit()                                
                                                                                
                                                                                

In [163]:
pool = ThreadPool(processes=4)
pool.map(failure_crawler,list(range(len(web_list_fl))))
pool.close()
pool.join()

Crawling google scholar website of Porf:Cristina Carbunaru
Crawling google scholar website of Porf:Erik Cambria
Crawling google scholar website of Porf:Cai Wentong
Performance AnalysisComputer NetworksSoftware Engineering
affective computingsentiment analysisnatural language understandingcommonsense reasoningsentic computing
Parallel and Distributed ComputingModeling and Simulation
Crawling google scholar website of Porf:Jie Zhang
Artificial IntelligenceIntelligent AgentTrustIncentiveUser Modeling
Successfully Crawling!
元素抓取报错
Crawling Failure!(Store failure info)
Crawling google scholar website of Porf:Anand BHOJAN
Wireless Networks & Security (IoT)AI/RoboticsVirtual Reality (Game) DesignCloud Computing
Successfully Crawling!
元素抓取报错
Crawling Failure!(Store failure info)
Successfully Crawling!
元素抓取报错
Crawling Failure!(Store failure info)
Successfully Crawling!
元素抓取报错
Crawling Failure!(Store failure info)
Crawling google scholar website of Porf:Arnab Bhattacharyya
Algorithmscomplexity
C

In [164]:
new_df_fail

,Name,Web
0,Erik Cambria,https://scholar.google.com/citations?user=ilSY...
1,Wooi-Boon Goh,https://scholar.google.com.sg/citations?user=w...
2,Cai Wentong,https://scholar.google.com/citations?user=QdmI...
3,Trevor Erik CARLSON,https://scholar.google.com/citations?user=bUK5...
4,Jie Zhang,https://scholar.google.com/citations?hl=en&use...
5,Arnab Bhattacharyya,https://scholar.google.com/citations?hl=en&use...
6,Cristina Carbunaru,https://scholar.google.com/citations?hl=en&use...
7,Anand BHOJAN,https://scholar.google.com/citations?hl=en&use...
8,Cham Tat Jen,https://scholar.google.com/citations?user=Lx3X...
9,Owen Noel Newton Fernando,https://scholar.google.com/citations?hl=en&use...


In [ ]:
new_df_fail.to_csv('failure.csv',index=False,header=False)
